# Debugging Strategic Resume Endpoint

This notebook helps debug issues with the strategic resume endpoint where agent responses are not properly populating the final output. We'll inspect raw responses, validate schemas, apply repairs, and ensure data flows correctly through the schema assembler.

## Section 1: Inspect Raw Agent Responses

Load and examine the raw JSON responses from agents for experiences, skills, projects, summary, design_brief, jinja_template, and css_styles.

In [ ]:
import json

# Raw agent responses from the logs
raw_responses = {
    "skills": """{
  "skills": [
    {
      "id": "skill_1",
      "name": "JavaScript",
      "level": 3
    },
    {
      "id": "skill_2",
      "name": "React",
      "level": 3
    },
    {
      "id": "skill_3",
      "name": "Front-end Development",
      "level": 3
    },
    {
      "id": "skill_4",
      "name": "HTML",
      "level": 3
    },
    {
      "id": "skill_5",
      "name": "CSS",
      "level": 3
    },
    {
      "id": "skill_6",
      "name": "UI",
      "level": 3
    },
    {
      "id": "skill_7",
      "name": "Web Development",
      "level": 3
    }
  ],
  "additional_skills": [
    "Agile",
    "Problem-solving",
    "Communication",
    "Collaboration",
    "Mentorship"
  ]
}""",
    "projects": """{
  "projects": [
    {
      "id": "proj_1",
      "name": "Cisco Prime Widget Library Redesign",
      "description": "Redesigned the Cisco Prime Widget Library in two months, focusing on mobile usability, clean design, and improved performance. Created an icon-font of over 280 icons, reducing icon file size footprint to 25%.",
      "url": null
    },
    {
      "id": "proj_2",
      "name": "Multimedia Animation and Visual Design",
      "description": "Produced original multimedia animations, shot product photos, edited videos for brand marketing, and worked on the visual aspects of promotional campaigns. Designed print materials and revitalized websites.",
      "url": null
    },
    {
      "id": "proj_3",
      "name": "Cinematic Animation for AAA Titles",
      "description": "Created stunning, responsive, cinematic animations for 3+ AAA titles. Collaborated with design, art, and engineering teams to define animation styles, implement solutions, troubleshoot, and debug animation systems.",
      "url": null
    }
  ]
}""",
    "experiences": """{
  "experiences": [
    {
      "id": "exp_1",
      "company": "General Motors",
      "position": "Multimedia Designer",
      "startDate": "2008",
      "endDate": "2010",
      "responsibilities": [
        "Produced original multimedia animations, shot product photos, edited videos for brand marketing, and worked on the visual of all promotional campaigns.",
        "Designed print materials, revitalized websites, and developed various digital elements for movies, television shows, and short videos.",
        "Communicated with managers."
      ]
    },
    {
      "id": "exp_2",
      "company": "Verizon",
      "position": "Character Animator",
      "startDate": "2011",
      "endDate": "2013",
      "responsibilities": [
        "Created stunning, responsive, cinematic animations for 3+ AAA titles.",
        "Worked closely with the design team to create immersive cinematic experiences.",
        "Implemented, troubleshot, and debugged animation systems.",
        "Collaborated with the art, engineering, and design teams to define the animation style for the project."
      ]
    },
    {
      "id": "exp_3",
      "company": "Intel",
      "position": "Layout Artist",
      "startDate": "2014",
      "endDate": "2015",
      "responsibilities": [
        "Designed promotion and information print pieces such as scientific posters, medical conference invitations, prescription drug brochures, and symposium newsletters.",
        "Creation of new website layouts for upcoming pharmaceutical drug education presentations.",
        "Assisted Creative Director in conceptualizing designs for upcoming projects."
      ]
    },
    {
      "id": "exp_4",
      "company": "Oracle Corporation",
      "position": "Art Director",
      "startDate": "2016",
      "endDate": "2017",
      "responsibilities": [
        "Supervise multi-functional project teams of 10+ colleagues to develop creative and effective advertising concepts, from ideation through final projects.",
        "Incorporate and explore client suggestions and directives, resolve questions and concerns, oversee objections.",
        "Organize all creative materials to ensure their smooth transition to other departments.",
        "Commission artwork; negotiate costs and usage, oversee completion of artwork, supervise photoshoots."
      ]
    },
    {
      "id": "exp_5",
      "company": "Tech Data",
      "position": "UX Designer",
      "startDate": "2017",
      "endDate": "2018",
      "responsibilities": [
        "Completed redesign of Cisco Prime Widget Library in only two months which consists of over 100 widgets focusing on mobile usability, clean design and improved performance.",
        "Created an icon-font which consisted of over 280 icons to replace the previous sprite image based system.",
        "By doing this icons remained crisp and clear at any size even on retina displays and reduced icon file size footprint to 25% of the previous version Heavily involved in every step of application."
      ]
    },
    {
      "id": "exp_6",
      "company": "Alliance Data",
      "position": "Production Artist",
      "startDate": "2019",
      "endDate": "2021",
      "responsibilities": [
        "Designed HTML supported e-mail marketing ads targeted to specific reader demographic populations.",
        "Evaluated production schedules and print specifications designed to foster greater accuracy, feasibility",
        "Presented effective design strategies intended to increase revenue and subscription membership",
        "Performed art/photo search, selection, and preparation of required materials for print or web use.",
        "Adapted to changing marketing plans Coordinated product design."
      ]
    }
  ]
}""",
    "summary": """{
  "summary": "A highly versatile front-end software engineer with a rich background in design and animation, adept at blending technical expertise with creative vision. Proven ability to lead and collaborate within cross-functional teams, delivering innovative solutions that enhance user experience and drive business growth. Expertise in front-end technologies such as JavaScript, React, HTML, and CSS, complemented by a strong foundation in UI/UX principles and a passion for creating visually stunning and highly functional web applications."
}""",
    "design_brief": """{
  "layout_description": "Two-column layout. Left column uses a solid background color for contact info, education, and skills. The right column contains a statement/summary, employment history, awards, and certifications. The name is prominently displayed at the top, spanning both columns. The right column uses subtle section dividers. Consistent use of icons to denote dates and locations.",
  "color_palette": {
    "primary": "#264653",
    "secondary": "#2A9D8F",
    "accent": "#E9C46A",
    "neutral": "#F4A261",
    "background": "#FFFFFF",
    "text": "#000000"
  },
  "google_fonts": [
    "Roboto",
    "Montserrat",
    "Open Sans"
  ],
  "design_prompt_for_developer": "Create a clean, modern resume template with a two-column layout. Use the provided color palette and fonts. Emphasize a strong visual hierarchy with clear section headings and consistent use of icons. The left column should have a solid background color and contain contact information, education, and skills. The right column should include a statement, employment history, awards, and certifications. Ensure the design is responsive and print-friendly."
}""",
    "jinja_template": """{
  "jinja_template": "{% extends 'base.html' %} {% block content %} <div class=\"container\"> <header> <h1>{{ name }}</h1> <p class=\"job-title\">{{ job_title }}</p> </header> <div class=\"main-content\"> <div class=\"left-column\"> <section class=\"contact\"> <h2>Contact</h2> <ul> <li><strong>Phone:</strong> {{ contact.phone }}</li> <li><strong>Email:</strong> {{ contact.email }}</li> <li><strong>LinkedIn:</strong> {{ contact.linkedin }}</li> <li><strong>GitHub:</strong> {{ contact.github }}</li> </ul> </section> <section class=\"education\"> <h2>Education</h2> {% for edu in education %} <div class=\"edu-item\"> <h3>{{ edu.degree }}, {{ edu.university }}</h3> <p class=\"date\">{{ edu.start_date }} - {{ edu.end_date }}</p> <p>{{ edu.description }}</p> </div> {% endfor %} </section> <section class=\"skills\"> <h2>Skills</h2> <ul> {% for skill in skills %} <li>{{ skill }}</li> {% endfor %} </ul> </section> </div> <div class=\"right-column\"> <section class=\"summary\"> <h2>Summary</h2> <p>{{ summary }}</p> </section> <section class=\"experience\"> <h2>Experience</h2> {% for exp in experience %} <div class=\"exp-item\"> <h3>{{ exp.position }}, {{ exp.company }}</h3> <p class=\"date\">{{ exp.start_date }} - {{ exp.end_date }}</p> <p class=\"description\">{{ exp.description }}</p> </div> {% endfor %} </section> <section class=\"projects\"> <h2>Projects</h2> {% for project in projects %} <div class=\"project-item\"> <h3>{{ project.name }}</h3> <p class=\"description\">{{ project.description }}</p> </div> {% endfor %} </section> </div> </div> </div> {% endblock %}",
  "css_styles": "@import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&family=Montserrat:wght@400;700&family=Open+Sans:wght@400;700&display=swap');\n\nbody {\n font-family: 'Open Sans', sans-serif;\n background-color: #FFFFFF;\n color: #000000;\n margin: 0;\n padding: 0;\n display: flex;\n justify-content: center;\n align-items: center;\n min-height: 100vh;\n box-sizing: border-box;\n }\n\n.container {\n width: 80%;\n max-width: 1200px;\n margin: 20px auto;\n background-color: #FFFFFF;\n box-shadow: 0 0 20px rgba(0, 0, 0, 0.1);\n display: grid;\n grid-template-columns: 30% 70%;\n gap: 20px;\n padding: 20px;\n }\n\nheader {\n grid-column: 1 / span 2;\n text-align: center;\n padding-bottom: 20px;\n border-bottom: 2px solid #E9C46A;\n }\n\nheader h1 {\n font-family: 'Montserrat', sans-serif;\n color: #264653;\n margin-bottom: 5px;\n }\n\nheader .job-title {\n font-style: italic;\n color: #2A9D8F;\n }\n\n.left-column {\n background-color: #264653;\n color: #FFFFFF;\n padding: 20px;\n }\n\n.left-column h2 {\n font-family: 'Roboto', sans-serif;\n color: #E9C46A;\n border-bottom: 1px solid #FFFFFF;\n padding-bottom: 10px;\n margin-bottom: 15px;\n }\n\n.left-column ul {\n list-style-type: none;\n padding: 0;\n }\n\n.left-column li {\n margin-bottom: 8px;\n }\n\n.right-column {\n padding: 20px;\n }\n\n.right-column h2 {\n font-family: 'Roboto', sans-serif;\n color: #264653;\n border-bottom: 1px solid #E9C46A;\n padding-bottom: 10px;\n margin-bottom: 15px;\n }\n\n.exp-item, .project-item {\n margin-bottom: 20px;\n }\n\n.exp-item h3, .project-item h3 {\n color: #2A9D8F;\n margin-bottom: 5px;\n }\n\n.date {\n font-style: italic;\n color: #777;\n margin-bottom: 10px;\n }\n\n.description {\n line-height: 1.6;\n }\n\nsection {\n margin-bottom: 30px;\n }\n\n/* Responsive Design */\n@media (max-width: 768px) {\n .container {\n grid-template-columns: 100%;\n }\n header {\n grid-column: 1;\n }\n}\n"
}"""
}

# Parse and inspect
parsed_responses = {}
for key, raw in raw_responses.items():
    try:
        parsed = json.loads(raw)
        parsed_responses[key] = parsed
        print(f"✅ {key}: Parsed successfully")
        print(f"   Type: {type(parsed)}")
        if isinstance(parsed, dict):
            print(f"   Keys: {list(parsed.keys())}")
        elif isinstance(parsed, list):
            print(f"   Length: {len(parsed)}")
        print()
    except json.JSONDecodeError as e:
        print(f"❌ {key}: Failed to parse - {e}")
        print(f"   Raw: {raw[:100]}...")
        print()

## Section 2: Parse and Validate JSON Outputs

Use Python's json module to parse the raw responses and validate them against Pydantic schemas like ExperienceAgentOutPutSchema, SkillsAgentOutPutSchema, etc.

In [ ]:
from app.schemas.ResumeSchemas import (
    ExperienceAgentOutPutSchema, SkillsAgentOutPutSchema, ProjectsAgentOutPutSchema,
    SummaryAgentOutPutSchema, DesignBriefOutputSchema, DesignerAgentOutputSchema
)

# Validate each parsed response against its schema
validation_results = {}
for key, parsed in parsed_responses.items():
    schema_map = {
        "experiences": ExperienceAgentOutPutSchema,
        "skills": SkillsAgentOutPutSchema,
        "projects": ProjectsAgentOutPutSchema,
        "summary": SummaryAgentOutPutSchema,
        "design_brief": DesignBriefOutputSchema,
        "jinja_template": DesignerAgentOutputSchema,
    }
    
    if key in schema_map:
        schema = schema_map[key]
        try:
            validated = schema(**parsed)
            validation_results[key] = {"status": "OK", "data": validated.model_dump()}
            print(f"✅ {key}: Validation passed")
        except Exception as e:
            validation_results[key] = {"status": "FAILED", "error": str(e)}
            print(f"❌ {key}: Validation failed - {e}")
    else:
        print(f"⚠️ {key}: No schema defined")
    print()

## Section 3: Debug Schema Validation Errors

Identify and debug errors such as 'argument after ** must be a mapping, not list' by checking data types and ensuring proper dictionary structures.

In [ ]:
# Debug validation errors
for key, result in validation_results.items():
    if result["status"] == "FAILED":
        print(f"🔍 Debugging {key}:")
        parsed = parsed_responses[key]
        print(f"   Parsed type: {type(parsed)}")
        if isinstance(parsed, dict):
            print(f"   Parsed keys: {list(parsed.keys())}")
            for k, v in parsed.items():
                print(f"     {k}: {type(v)} - {v if not isinstance(v, list) or len(v) <= 2 else f'list of {len(v)} items'}")
        else:
            print(f"   Parsed value: {parsed}")
        print(f"   Error: {result['error']}")
        
        # Try to identify the issue
        if "argument after ** must be a mapping" in result["error"]:
            print("   Issue: Trying to unpack a non-dict with **")
            if isinstance(parsed, list):
                print("   Solution: Wrap list in dict with appropriate key")
        print()
    else:
        print(f"✅ {key}: No issues")
        print()

## Section 4: Implement Schema Assembler Repairs

Code repairs in schema_assembler.py to handle coercions, wrap non-dict data, and apply defaults for missing fields like layout_description and color_palette.

In [ ]:
from app.agents.resume.strategic.schema_assembler import SchemaAssembler

# Simulate fragments as they would be in the code
fragments = {
    "experiences": parsed_responses.get("experiences", {}).get("experiences", []),
    "skills": parsed_responses.get("skills", {}),
    "projects": parsed_responses.get("projects", {}).get("projects", []),
    "education": {"education": []},  # From resume
    "contact_info": {"contact_info": []},  # From resume
    "summary": parsed_responses.get("summary", {}).get("summary", ""),
    "design_brief": parsed_responses.get("design_brief", {}),
    "jinja_template": parsed_responses.get("jinja_template", {}),
    "css_styles": ""
}

print("Original fragments:")
for k, v in fragments.items():
    print(f"  {k}: {type(v)} - {v if not isinstance(v, (list, dict)) or len(str(v)) <= 50 else f'{type(v)} of length {len(v) if isinstance(v, (list, dict)) else len(str(v))}'}")

print("\nRunning schema assembler...")
assembler = SchemaAssembler()
final_resume, diagnostics = assembler.assemble_resume_object(fragments)

print("\nDiagnostics:")
for diag in diagnostics:
    if diag.status == "FAILED":
        print(f"❌ {diag.field}: {diag.error_message}")
    elif diag.repairs_applied:
        print(f"⚠️ {diag.field}: {', '.join(diag.repairs_applied)}")
    else:
        print(f"✅ {diag.field}: OK")

print("\nFinal resume keys:", list(final_resume.keys()))
print("Experiences populated:", len(final_resume.get("experiences", [])) > 0)
print("Skills populated:", len(final_resume.get("skills", [])) > 0)
print("Projects populated:", len(final_resume.get("projects", [])) > 0)

## Section 5: Merge Agent Outputs into Final Schema

Combine validated outputs from all agents into a single Pydantic object, ensuring arrays like experiences and skills are populated correctly.

In [ ]:
# The assembler already merges them, but let's inspect the final result
print("Final merged resume:")
for key in ["experiences", "skills", "projects", "summary", "design_brief", "jinja_template"]:
    value = final_resume.get(key)
    if isinstance(value, list):
        print(f"  {key}: {len(value)} items")
        if value and len(value) > 0:
            print(f"    Sample: {value[0] if len(str(value[0])) <= 100 else str(value[0])[:100] + '...'}")
    elif isinstance(value, dict):
        print(f"  {key}: {len(value)} keys - {list(value.keys())}")
    else:
        print(f"  {key}: {value if len(str(value)) <= 100 else str(value)[:100] + '...'}")
    print()

# Verify the structure matches expected output
expected_keys = ["experiences", "skills", "projects", "education", "contact_info", "summary", "design_brief", "jinja_template", "css_styles"]
missing_keys = [k for k in expected_keys if k not in final_resume]
if missing_keys:
    print(f"❌ Missing keys: {missing_keys}")
else:
    print("✅ All expected keys present")

## Section 6: Test Assembler with Sample Data

Run unit tests on the assembler using sample raw responses to verify that empty arrays are filled and validation passes.

In [ ]:
# Test with edge cases
test_cases = [
    # Case 1: Normal case (already tested)
    {
        "name": "Normal case",
        "fragments": fragments
    },
    # Case 2: Empty responses
    {
        "name": "Empty responses",
        "fragments": {
            "experiences": [],
            "skills": {},
            "projects": [],
            "education": {"education": []},
            "contact_info": {"contact_info": []},
            "summary": "",
            "design_brief": {},
            "jinja_template": {},
            "css_styles": ""
        }
    },
    # Case 3: Malformed data
    {
        "name": "Malformed skills",
        "fragments": {
            "experiences": [{"id": "exp_1", "company": "Test"}],
            "skills": {"skills": "not a list"},
            "projects": [],
            "education": {"education": []},
            "contact_info": {"contact_info": []},
            "summary": "Test",
            "design_brief": {"layout_description": "test"},
            "jinja_template": {"jinja_template": "test"},
            "css_styles": ""
        }
    }
]

for test_case in test_cases:
    print(f"🧪 Testing: {test_case['name']}")
    assembler = SchemaAssembler()
    final, diags = assembler.assemble_resume_object(test_case['fragments'])
    
    failed_diags = [d for d in diags if d.status == "FAILED"]
    if failed_diags:
        print(f"   ❌ {len(failed_diags)} failed validations")
        for d in failed_diags:
            print(f"      {d.field}: {d.error_message}")
    else:
        print("   ✅ All validations passed")
    
    # Check if arrays are populated
    populated = []
    for key in ["experiences", "skills", "projects"]:
        if len(final.get(key, [])) > 0:
            populated.append(key)
    print(f"   📊 Populated arrays: {populated}")
    print()

## Section 7: Handle Edge Cases in Validation

Address issues like failed JSON parsing for strings (e.g., summary) and ensure all required fields are present before final output.

In [ ]:
# Handle edge cases
edge_cases = {
    "summary_as_string": "This is a plain string summary without JSON wrapper",
    "summary_malformed_json": '{"summary": "Valid JSON but test edge"}',
    "design_brief_missing_fields": {"layout_description": "test"},
    "jinja_template_as_string": "Plain Jinja template string",
}

print("🔧 Testing edge case handling:")
for case_name, value in edge_cases.items():
    print(f"  {case_name}: {type(value)} - {str(value)[:50]}...")
    
    # Simulate normalize_input
    if isinstance(value, str):
        try:
            normalized = json.loads(value)
            print("    ✅ Parsed as JSON")
        except:
            normalized = value
            print("    ⚠️ Treated as plain string")
    else:
        normalized = value
    
    # Simulate wrapping
    if not isinstance(normalized, dict):
        wrapped = {"value": normalized}  # Simplified wrap
        print(f"    🔄 Wrapped: {wrapped}")
    else:
        wrapped = normalized
        print("    ✅ Already dict")
    
    print()

print("📋 Summary of fixes applied:")
print("1. Added wrapping for non-dict inputs in schema_assembler.py")
print("2. Made coercion dynamic based on schema fields")
print("3. Fixed Pydantic v1/v2 compatibility")
print("4. Simplified extraction logic to handle dict and non-dict cases")
print("5. Ensured all required fields are present in final output")
print("\n🎯 These changes should resolve the empty arrays issue in the strategic endpoint.")